In [1]:
import pandas as pd
train = pd.read_csv('train.csv',index_col=0)
train.head(3)

,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label
id,,,,,,,
0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated
3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated
1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated


In [2]:
cols = ['title1_en','title2_en','label']
train = train.loc[:, cols]
train.head(3)

,title1_en,title2_en,label
id,,,
0,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated
3,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated
1,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated


In [3]:
import string
def en_tokenizer(text):
    tokens = text.translate(str.maketrans('', '', string.punctuation))
    return tokens

In [4]:
train['title1_tokenized'] = train.loc[:, 'title1_en'].apply(en_tokenizer)
train['title2_tokenized'] = train.loc[:, 'title2_en'].apply(en_tokenizer)
train.iloc[:, [0, 3]].head()

,title1_en,title1_tokenized
id,,
0,There are two new old-age insurance benefits f...,There are two new oldage insurance benefits fo...
3,"""If you do not come to Shenzhen, sooner or lat...",If you do not come to Shenzhen sooner or later...
1,"""If you do not come to Shenzhen, sooner or lat...",If you do not come to Shenzhen sooner or later...
2,"""If you do not come to Shenzhen, sooner or lat...",If you do not come to Shenzhen sooner or later...
9,"""How to discriminate oil from gutter oil by me...",How to discriminate oil from gutter oil by mea...


In [5]:
train.iloc[:, [1, 4]].head()

,title2_en,title2_tokenized
id,,
0,"Police disprove ""bird's nest congress each per...",Police disprove birds nest congress each perso...
3,Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,Shenzhens GDP outstrips Hong Kong Shenzhen Sta...
1,The GDP overtopped Hong Kong? Shenzhen clarifi...,The GDP overtopped Hong Kong Shenzhen clarifie...
2,Shenzhen's GDP topped Hong Kong last year? She...,Shenzhens GDP topped Hong Kong last year Shenz...
9,It took 30 years of cooking oil to know that o...,It took 30 years of cooking oil to know that o...


In [6]:
from tensorflow import keras
MAX_NUM_WORDS = 35000
tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS)

In [7]:
corpus_x1 = train.title1_tokenized
corpus_x2 = train.title2_tokenized
corpus = pd.concat([corpus_x1, corpus_x2])
corpus.shape

(641104,)

In [8]:
pd.DataFrame(corpus.iloc[:5], columns=['title'])

,title
id,
0,There are two new oldage insurance benefits fo...
3,If you do not come to Shenzhen sooner or later...
1,If you do not come to Shenzhen sooner or later...
2,If you do not come to Shenzhen sooner or later...
9,How to discriminate oil from gutter oil by mea...


In [9]:
tokenizer.fit_on_texts(corpus)

In [10]:
x1_train = tokenizer.texts_to_sequences(corpus_x1)
x2_train = tokenizer.texts_to_sequences(corpus_x2)
len(x1_train)

320552

In [11]:
x1_train[:1]

[[90, 17, 58, 27, 1285, 561, 788, 12, 64, 30, 7, 87, 426, 20, 8, 410, 174]]

In [12]:
for seq in x1_train[:1]:
    print([tokenizer.index_word[idx] for idx in seq])

['there', 'are', 'two', 'new', 'oldage', 'insurance', 'benefits', 'for', 'old', 'people', 'in', 'rural', 'areas', 'have', 'you', 'got', 'them']


In [13]:
for seq in x1_train[:10]:
    print(len(seq),seq[:5],' ...')

17 [90, 17, 58, 27, 1285]  ...
28 [112, 8, 21, 9, 194]  ...
28 [112, 8, 21, 9, 194]  ...
28 [112, 8, 21, 9, 194]  ...
11 [33, 2, 4323, 371, 44]  ...
28 [112, 8, 21, 9, 194]  ...
13 [112, 8, 25, 2407, 8]  ...
28 [112, 8, 21, 9, 194]  ...
10 [1065, 1065, 37, 4, 2699]  ...
11 [33, 2, 4323, 371, 44]  ...


In [14]:
max([len(seq) for seq in x1_train])

401

In [15]:
MAX_SEQUENCE_LENGTH = 300
x1_train = keras.preprocessing.sequence.pad_sequences(x1_train, maxlen=MAX_SEQUENCE_LENGTH)
x2_train = keras.preprocessing.sequence.pad_sequences(x2_train, maxlen=MAX_SEQUENCE_LENGTH)

In [16]:
train.label[:5]

id
0    unrelated
3    unrelated
1    unrelated
2    unrelated
9       agreed
Name: label, dtype: object

In [17]:
import numpy as np

label_to_index = {'unrelated': 0, 'agreed': 1, 'disagreed': 2}
y_train = train.label.apply(lambda x: label_to_index[x])
y_train = np.asarray(y_train).astype('float32')

In [18]:
y_train[:10]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 1.], dtype=float32)

In [19]:
y_train = keras.utils.to_categorical(y_train)
y_train[:10]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [20]:
from sklearn.model_selection import train_test_split

VALIDATION_RATIO = 0.1
RANDOM_STATE = 1000

x1_train, x1_val, x2_train, x2_val, y_train, y_val =\
train_test_split(x1_train, x2_train, y_train, test_size=VALIDATION_RATIO, random_state=RANDOM_STATE)

In [21]:
print('Training Set')
print('-' * 10)
print(f'x1_train: {x1_train.shape}')
print(f'x2_train: {x2_train.shape}')
print(f'y_train : {y_train.shape}')
print('-' * 10)
print(f'x1_val:   {x1_val.shape}')
print(f'x2_val:   {x2_val.shape}')
print(f'y_val :   {y_val.shape}')
print('-' * 10)
print('Test Set')

Training Set
----------
x1_train: (288496, 300)
x2_train: (288496, 300)
y_train : (288496, 3)
----------
x1_val:   (32056, 300)
x2_val:   (32056, 300)
y_val :   (32056, 3)
----------
Test Set


In [22]:
for i, seq in enumerate(x1_train[:3]):
    print(f'News Title {i}: ')
    print(seq)
    print()

News Title 0: 
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0  

In [23]:
for i, seq in enumerate(x1_train[:3]):
    print(f'News Title {i}: ')
    print([tokenizer.index_word.get(idx, '') for idx in seq])
    print()

News Title 0: 
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 

In [24]:
NUM_CLASSES = 3
MAX_NUM_WORDS = 35000
MAX_SEQUENCE_LENGTH = 300
NUM_EMBEDDING_DIM = 256
NUM_LSTM_UNITS = 128

In [25]:
from tensorflow.keras import Input
from tensorflow.keras.layers import Embedding, LSTM, concatenate, Dense
from tensorflow.keras.models import Model

top_input = Input(shape=(MAX_SEQUENCE_LENGTH, ), dtype='int32')
buttom_input = Input(shape=(MAX_SEQUENCE_LENGTH, ), dtype='int32')

embedding_layer = Embedding(MAX_NUM_WORDS, NUM_EMBEDDING_DIM)
top_embedding = embedding_layer(top_input)
buttom_embedding = embedding_layer(buttom_input)

lstm_layer = LSTM(NUM_LSTM_UNITS)
top_output = lstm_layer(top_embedding)
buttom_output = lstm_layer(buttom_embedding)

merged = concatenate([top_output, buttom_output], axis=1)

fc_layer = Dense(NUM_CLASSES, 'softmax')
predictions = fc_layer(merged)

model = Model(inputs=[top_input, buttom_input], outputs=predictions)

In [26]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 300)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 300, 256)     8960000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 128)          197120      embedding[0][0]       

In [27]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
BATCH_SIZE = 1024
NUM_EPOCHS = 30

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=2, mode='max')
checkpoint = ModelCheckpoint(filepath='model.{epoch:02d}.h5')

history = model.fit(x=[x1_train, x2_train],\
    y=y_train,\
    batch_size=BATCH_SIZE,\
    epochs=NUM_EPOCHS,\
    validation_data=([x1_val, x2_val], y_val),\
    shuffle=True,\
    callbacks=[early_stopping, checkpoint])

In [28]:
test = pd.read_csv('test.csv',index_col=0)

test['title1_tokenized'] = test.loc[:, 'title1_en'].apply(en_tokenizer)
test['title2_tokenized'] = test.loc[:, 'title2_en'].apply(en_tokenizer)

x1_test = tokenizer.texts_to_sequences(test.title1_tokenized)
x2_test = tokenizer.texts_to_sequences(test.title2_tokenized)

x1_test = keras.preprocessing.sequence.pad_sequences(x1_test, maxlen=MAX_SEQUENCE_LENGTH)
x2_test = keras.preprocessing.sequence.pad_sequences(x2_test, maxlen=MAX_SEQUENCE_LENGTH)

model = keras.models.load_model('model.08.h5')
predictions = model.predict([x1_test, x2_test])

predictions[:10]

array([[9.9999964e-01, 3.5797012e-08, 4.0200655e-07],
       [9.9947995e-01, 3.3637854e-05, 4.8651820e-04],
       [9.9998701e-01, 9.5523528e-06, 3.4135503e-06],
       [9.9990809e-01, 1.3599135e-06, 9.0639289e-05],
       [9.9997616e-01, 7.7560124e-08, 2.3732495e-05],
       [3.9104998e-01, 6.0886270e-01, 8.7410051e-05],
       [9.9995720e-01, 1.2388745e-05, 3.0361736e-05],
       [2.4140307e-01, 7.5855392e-01, 4.3067128e-05],
       [4.0024039e-01, 5.9956914e-01, 1.9040714e-04],
       [9.0300465e-01, 9.4931506e-02, 2.0638276e-03]], dtype=float32)

In [29]:
index_to_label = {v: k for k, v in label_to_index.items()}

test['Category'] = [index_to_label[idx] for idx in np.argmax(predictions, axis=1)]

submission = test.loc[:, ['Category']]
# submission.columns = ['Id', 'Category']
print(submission.shape)
submission.head()

(80126, 1)


,Category
id,
321187,unrelated
321190,unrelated
321189,unrelated
321193,unrelated
321191,unrelated


In [30]:
submission.to_csv('submission.csv')